In [77]:
#'cffex-host-alarm.csv' process code
#Author: 普俊韬
#Last update: 20180626

#导入pandas、numpy库
import pandas as pd
import numpy as np
import re


#TODO：进行原始数据的分割处理

#读取原始数据（GBK编码，数据不带列标题）
data = pd.read_csv('raw_data\cffex-host-alarm\cffex-host-alarm.csv',encoding = 'GBK', header = None)

#分割数据字段，分隔符'|||'（'[|]+'),分割后扩展列
data_processed = data[0].str.split('[|]+',expand = True)

#插入列标题
data_processed.columns = ['node_name', 'node_alias', 'component', 'category', 'alarm_count', 'first_time', 'last_time', 'alarm_level', 'alarm_content']


#TODO：进行'component'字段的处理

#将'component'字段提取出来作为一个DataFrame
data_component = data_processed.loc[:,['component']]

#去掉重复数据
data_component_processed = data_component.drop_duplicates()

#插入id列，编号从1开始
data_component_processed['id'] = range(1,len(data_component_processed) + 1)

#将列顺序调整为['id', 'component']
data_component_processed = data_component_processed[['id','component']]

#将处理后结果写入'cffex-host-alarm-component.csv'（不带行标签，GBK编码）
data_component_processed.to_csv('output_data\cffex-host-alarm\cffex-host-alarm-component.csv', index = 0, encoding = 'GBK')


#TODO：进行'category'字段的处理

#将'category'字段提取出来作为一个DataFrame
data_category = data_processed.loc[:,['category']]

#去掉重复数据
data_category_processed = data_category.drop_duplicates()

#插入id列，编号从1开始
data_category_processed['id'] = range(1,len(data_category_processed) + 1)

#将列顺序调整为['id', 'category']
data_category_processed = data_category_processed[['id','category']]

#将处理后结果写入'cffex-host-alarm-category.csv'（不带行标签，GBK编码）
data_category_processed.to_csv('output_data\cffex-host-alarm\cffex-host-alarm-category.csv', index = 0, encoding = 'GBK')


#TODO：进行'node_alias'字段的处理

#将'node_alias'字段提取出来作为一个DataFrame
data_node_alias = data_processed.loc[:,['node_alias']]

#去掉重复数据
data_node_alias_processed = data_node_alias.drop_duplicates()

#插入id列，编号从1开始
data_node_alias_processed['id'] = range(1,len(data_node_alias_processed) + 1)

#将列顺序调整为['id', 'node_alias']
data_node_alias_processed = data_node_alias_processed[['id','node_alias']]

#将处理后结果写入'cffex-host-alarm-node-alias.csv'（不带行标签，GBK编码）
data_node_alias_processed.to_csv('output_data\cffex-host-alarm\cffex-host-alarm-node-alias.csv', index = 0, encoding = 'GBK')


#TODO：将'component','category'和'node_alias'字段替换为对应的'id'值，方便后续的数据处理

#对'component'字段进行查找和替换
data_processed['component'] = data_processed['component'].replace(data_component_processed['component'].tolist(),data_component_processed['id'].tolist())

#对'category'字段进行查找和替换
data_processed['category'] = data_processed['category'].replace(data_category_processed['category'].tolist(),data_category_processed['id'].tolist())

#对'node_alias'字段进行查找和替换
data_processed['node_alias'] = data_processed['node_alias'].replace(data_node_alias_processed['node_alias'].tolist(),data_node_alias_processed['id'].tolist())


#TODO: 将'alarm_content'字段替换为相应的'id'值，方便后续的数据处理

#读入告警事件表'cffex-host-alarm-content.csv'
data_processed_content = pd.read_csv('raw_data\cffex-host-alarm\cffex-host-alarm-content.csv',encoding = 'GBK')

#替换函数定义
def re_replace(data):
    for i in range(len(data_processed_content['id'])):
        #正则表达式和字符串不存在匹配串：继续遍历
        if re.match(data_processed_content['regular_expression'][i],data) == None:
            continue
        #正则表达式和字符串存在匹配串：替换id值并返回保存
        else:
            data = str(data_processed_content['id'][i])
            return data

#调用替换函数
data_processed['alarm_content'] = data_processed['alarm_content'].apply(re_replace)

#将处理后结果写入'cffex-host-alarm-processed.csv'（不带行标签，GBK编码）
data_processed.to_csv('output_data\cffex-host-alarm\cffex-host-alarm-processed.csv', index = 0, encoding = 'GBK')



e:\python37\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
e:\python37\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
e:\python37\lib\site-packages\ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0                      wgq_fibgate2_2(10.30.8.*)PING 连通性正常
1                      wgq_fibgate2_1(10.30.7.*)PING 连通性正常
2                      wgq_fibgate1_2(10.30.8.*)PING 连通性正常
3                      wgq_fibgate1_1(10.30.7.*)PING 连通性正常
4                         wgq_fibgate2_2(10.30.8.*)PING 不通
5                         wgq_fibgate1_2(10.30.8.*)PING 不通
6                         wgq_fibgate2_1(10.30.7.*)PING 不通
7                         wgq_fibgate1_1(10.30.7.*)PING 不通
8                          vcsdb1系统CPU使用率正常,应该小于70,实际为37.9
9                vcsdb1系统CPU使用率异常,应该小于70,实际为82.0,请联系主机岗处理。
10                        wgq_riskdb1(10.26.3.*)PING 连通性正常
11                        wgq_config3(10.26.1.*)PING 连通性正常
12                        wgq_config1(10.26.1.*)PING 连通性正常
13                           wgq_config1(10.26.1.*)PING 不通
14                        web2系统进程ProcessThcount-22485事件恢复
15       web1系统用户trade进程/cffex/deploy/apache2//bin/http...
16                      vcenter-wg1(10.22.240.*)PING 连通性